In [ ]:
# For a jupyter notebook add the following line:
%matplotlib inline
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import matplotlib.pyplot as plt

import psutil
import os

GA PART

In [ ]:
from statistics import median
from ga_model import*
from ga_process import*
from ga_load import*

# Collecting Data

Un des principaux objectifs de la modélisation basée sur les agents est de générer des données pour l'analyse
=> Mesa fournit une classe qui peut gérer la collecte et le stockage des données.

Le collecteur de données (Data Collector) stocke trois catégories de données : 
- les variables de niveau modèle, 
- les variables de niveau agent
- les tableaux. 

Les variables de niveau modèle et agent sont ajoutées au collecteur de données avec une fonction permettant de les collecter. Les fonctions de collecte au niveau du modèle prennent un objet modèle en entrée, tandis que les fonctions de collecte au niveau de l'agent prennent un objet agent en entrée. 

Elles renvoient ensuite une valeur calculée à partir du modèle ou de chaque agent dans leur état actuel. Lorsque la méthode collect du collecteur de données est appelée, avec un objet modèle comme argument, elle applique chaque fonction de collecte de niveau modèle au modèle, et stocke les résultats dans un dictionnaire, en associant la valeur actuelle à l'étape actuelle du modèle. De la même manière, la méthode applique chaque fonction de collecte de niveau agent à chaque agent actuellement dans le planning, en associant la valeur résultante à l'étape du modèle et à l'unique_id de l'agent.

Ajoutons un DataCollector au modèle, et collectons deux variables. Au niveau de l'agent, nous voulons collecter la richesse de chaque agent à chaque étape. Au niveau du modèle, mesurons le coefficient de **Gini du modèle**, une mesure de l'inégalité de la richesse.

In [ ]:
def compute_gini(model):
    agent_wealths = [agent.fitness for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    #La méthode Enumerate() ajoute un compteur à un itérable et le renvoie sous la forme d'un objet d'énumération.
    return 1 + (1 / N) - 2 * B

In [ ]:
class MoneyAgent(Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
        self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other_agent = self.random.choice(cellmates)
            other_agent.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()

class genetic_agent(Agent):
    def __init__(self,unique_id,model,num_generation,population,rate_mutation,num_parent,num_pop,chromosome_modele=customers['CUSTOMER_CODE'].unique().tolist()):
        super().__init__(unique_id, model)
        self.wealth=1
        self.num_generation=num_generation
        self.population=population
        self.rate_mutation=rate_mutation
        self.num_parent=num_parent
        self.num_pop=num_pop
        self.chromosome_modele=chromosome_modele

        self.vrp=VRP(load_customers(customers),load_vehicle(vehicles,vehicles['VEHICLE_CODE'].unique()))
        self.modele_genetic=Modele_genetic(self.chromosome_modele,len(self.chromosome_modele))
        self.vrp_ga=VRP_GA(self.modele_genetic,self.num_generation,self.population,self.rate_mutation,self.num_parent,self.num_pop,self.chromosome_modele,self.vrp)
        self.population=self.vrp_ga.initialize_population(self.num_pop,self.chromosome_modele)  
        self.fitness=float('-inf')


        for i in range(num_generation):
            self.population=self.vrp_ga.evolution(self.modele_genetic,self.num_parent,self.population,self.rate_mutation,self.num_pop,self.chromosome_modele)
        for i in self.population:
            self.fitness=max(self.fitness,self.modele_genetic.fitness(i))

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
        self.pos, moore=True, include_center=False
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)
        # self.population=self.vrp_ga.evolution(self.modele_genetic,self.num_parent,self.population,self.rate_mutation,self.num_pop,self.chromosome_modele)
    def step(self):
        self.move()
        

class MoneyModel(Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        # Create agents
        for i in range(self.num_agents):
            a = genetic_agent(i,self,5,[],0.05,4,10)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            

                 #data Collection
        self.datacollector = DataCollector(
            model_reporters={"Gini": compute_gini}, agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

## Execution du modèle
À chaque étape du modèle, le collecteur de données collectera et stockera le coefficient de Gini actuel au niveau du modèle, ainsi que la richesse de chaque agent, en les associant à l'étape actuelle.

Nous exécutons le modèle.   
Le DataCollector peut exporter les données qu'il a collectées sous forme de DataFrame pandas, pour une analyse interactive facile.

In [ ]:
model = MoneyModel(5, 10, 10)
for i in range(3):
    model.step()

## Pour obtenir la série de coefficients de Gini sous forme de DataFrame pandas :

In [ ]:
gini = model.datacollector.get_model_vars_dataframe()
gini.plot()

In [ ]:
agent_wealth = model.datacollector.get_agent_vars_dataframe()
agent_wealth.plot()

Gestion d'espace

In [ ]:
print(u'space usage：%.4f GB' % (psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024 / 1024) )


In [ ]:
end_wealth = agent_wealth.xs(99, level="Step")["Wealth"]
end_wealth.hist(bins=range(agent_wealth.Wealth.max() + 1))

- L'index du DataFrame est constitué de **paires d'étapes de modèle et d'ID d'agent**.   
- Vous pouvez l'analyser comme vous le feriez pour n'importe quel autre DataFrame. 
Par exemple, pour obtenir un histogramme de la richesse des agents à la fin du modèle :

# Agents sur la grille

In [ ]:
import numpy as np

agent_counts = np.zeros((model.grid.width, model.grid.height))
for cell in model.grid.coord_iter():
    cell_content, x, y = cell
    agent_count = len(cell_content)
    agent_counts[x][y] = agent_count
plt.imshow(agent_counts, interpolation="nearest")
plt.colorbar()

# If running from a text editor or IDE, remember you'll need the following:
# plt.show()
    

In [ ]:

plt.show()



agent_wealth = [a.wealth for a in model.schedule.agents]
plt.hist(agent_wealth)

